In [1]:
import os

DATA_PATH = "Resampling_Data"
OUTPUT_PATH = "Padding_Data"
os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_PATH, "train"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_PATH, "val"), exist_ok=True)
TRAIN_DATA_PATH = os.path.join(DATA_PATH, "train")
VAL_DATA_PATH = os.path.join(DATA_PATH, "val")

In [2]:
import numpy as np
import nibabel as nib
import json

def pad_to_multiple(array, multiple=32, mode='constant', value=0):
    """
    Pad array so that each spatial dimension is a multiple of `multiple`.
    Returns padded array and padding info: [(pad_before, pad_after), ...]
    """
    shape = array.shape
    pad_width = []
    
    for dim in shape:
        total_pad = (multiple - dim % multiple) % multiple
        pad_before = total_pad // 2
        pad_after = total_pad - pad_before
        pad_width.append((pad_before, pad_after))

    padded = np.pad(array, pad_width, mode=mode, constant_values=value)
    return padded, pad_width

def update_metadata_with_padding(metadata, pad_width):
    metadata['padding'] = pad_width
    return metadata

def process_and_pad(dir_name, image_path, mask_path, metadata_path, out_path, padding_multiple=32):
    os.makedirs(out_path, exist_ok=True)
    # Load data
    image = np.load(image_path)
    mask = np.load(mask_path)
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)

    # Padding
    padded_image, pad_width_img = pad_to_multiple(image, multiple=padding_multiple, mode='constant', value=0)
    padded_mask, pad_width_mask = pad_to_multiple(mask, multiple=padding_multiple, mode='constant', value=0)

    assert pad_width_img == pad_width_mask, "Image and mask padding must match"

    # Update metadata
    updated_metadata = update_metadata_with_padding(metadata, pad_width_img)

    # Save
    # print(padded_image.shape)
    # print(padded_image.shape)
    # print(padded_image.nbytes / 1024**3, "GB")  # 打印大小（单位：GB）
    # basename = os.path.splitext(os.path.basename(image_path))[0].replace("_resampled", "")
    # test_array = np.zeros((100, 100, 100), dtype=np.float32)
    # np.save(os.path.join(out_path, "test.npy"), test_array)

    np.save(os.path.join(out_path, "GED4.npy"), padded_image)
    np.save(os.path.join(out_path, "mask_GED4.npy"), padded_mask)

    with open(os.path.join(out_path, "metadata.json"), "w") as f:
        json.dump(updated_metadata, f, indent=4)

    print(f"✅ Padded data saved to {out_path}")


In [3]:
dir_list = os.listdir(TRAIN_DATA_PATH)
for idx, dir_name in enumerate(dir_list):
    if not os.path.isdir(os.path.join(TRAIN_DATA_PATH, dir_name)):
        continue
    tmp_GED4 = os.path.join(TRAIN_DATA_PATH, dir_name, "GED4.npy")
    tmp_mask = os.path.join(TRAIN_DATA_PATH, dir_name, "mask_GED4.npy")
    tmp_metadata = os.path.join(TRAIN_DATA_PATH, dir_name, "metadata.json")
    
    output_dir = os.path.join(OUTPUT_PATH, "train", dir_name)
    padded_GED4 = process_and_pad(dir_name, tmp_GED4, tmp_mask, tmp_metadata, output_dir, padding_multiple=32)

dir_list = os.listdir(VAL_DATA_PATH)
for idx, dir_name in enumerate(dir_list):
    if not os.path.isdir(os.path.join(VAL_DATA_PATH, dir_name)):
        continue
    tmp_GED4 = os.path.join(VAL_DATA_PATH, dir_name, "GED4.npy")
    tmp_mask = os.path.join(VAL_DATA_PATH, dir_name, "mask_GED4.npy")
    tmp_metadata = os.path.join(VAL_DATA_PATH, dir_name, "metadata.json")
    
    output_dir = os.path.join(OUTPUT_PATH, "val", dir_name)
    padded_GED4 = process_and_pad(dir_name, tmp_GED4, tmp_mask, tmp_metadata, output_dir, padding_multiple=32)

    
    

FileNotFoundError: [Errno 2] No such file or directory: 'Resampling_Data/train'

In [ ]:
# from utils import Resampling_back
import json
import os
import numpy as np
from torch import tensor

Test_Path = "/data/hdc/jincan/long_seg/MICCAI25/Resampling/val"
dir_list = os.listdir(Test_Path)
for idx in range(len(dir_list)):
    image_path = os.path.join(Test_Path, dir_list[idx], "GED4.npy")
    image_np = np.load(image_path)
    mask_path = os.path.join(Test_Path, dir_list[idx], "mask_GED4.npy")
    mask_np = np.load(mask_path)
    print(f"The original shape is {image_np.shape}, {mask_np.shape}")
    metadata_path = os.path.join(Test_Path, dir_list[idx], "metadata.json")
    metadata = json.load(open(metadata_path, "r"))
    image_np = np.expand_dims(image_np, axis=0)  # Add batch dimension
    original_spacing = [metadata["original_spacing"]]
    print(f"The original spacing is {original_spacing}")
    original_origin = [metadata["original_origin"]]
    original_direction = [metadata["original_direction"]]
    padding_info = [metadata["padding_info"]]

    # onehot = np.eye(2)[image_np]       # shape: [D, H, W, C]
    # onehot = onehot.transpose(3, 0, 1, 2)
    image_np = tensor(image_np)  # Convert to tensor if using PyTorch, otherwise keep as numpy array
    image_np = Resampling_back(image_np, original_spacing, original_origin, original_direction, padding_info)
    print(f"The resampled shape is {image_np.shape}")

The original shape is (180, 240, 380), (64, 520, 640)
The original spacing is [[0.5938000082969666, 0.5938000082969666, 3.5]]
[[[4, 4], [1, 1], [0, 0]]]
Output shape after removing padding: torch.Size([172, 238, 380])
The resampled shape is torch.Size([2, 172, 238, 380])
The original shape is (180, 240, 380), (72, 260, 320)
The original spacing is [[1.1875, 1.1875, 3.0]]
[[[7, 7], [1, 1], [0, 0]]]
Output shape after removing padding: torch.Size([166, 238, 380])
The resampled shape is torch.Size([2, 166, 238, 380])
The original shape is (160, 240, 380), (64, 260, 320)
The original spacing is [[1.1875, 1.1875, 3.0]]
[[[6, 6], [1, 1], [0, 0]]]
Output shape after removing padding: torch.Size([148, 238, 380])
The resampled shape is torch.Size([2, 148, 238, 380])
The original shape is (200, 240, 380), (80, 260, 320)
The original spacing is [[1.1875, 1.1875, 3.0]]
[[[7, 8], [1, 1], [0, 0]]]
Output shape after removing padding: torch.Size([185, 238, 380])
The resampled shape is torch.Size([2, 

In [8]:
import SimpleITK as sitk
import torch

def Prediction_to_nii(output_dict, target_spacing, original_origin, original_direction):
    np_array = output_dict.cpu().numpy()  # Convert to numpy array

    if (np_array.ndim == 4):
        np_array = np.argmax(np_array, axis=0)  # Convert to class labels if needed
    elif (np_array.ndim == 3):
        np_array = np_array

    image = sitk.GetImageFromArray(np_array.astype(np.uint8))  # Convert to SimpleITK image
    image.SetSpacing(target_spacing)  # Set target spacing
    image.SetOrigin(original_origin)  # Set original origin
    image.SetDirection(original_direction)  # Set original direction
    return image  # Return the SimpleITK image

def resample_image(itk_image, out_spacing, is_label=False):
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()
    original_direction = itk_image.GetDirection()
    original_origin = itk_image.GetOrigin()

    out_size = [
        int(round(osz * ospc / nspc))
        for osz, ospc, nspc in zip(itk_image.GetSize(), original_spacing, out_spacing)
    ]

    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(out_spacing)
    resampler.SetSize(out_size)
    resampler.SetOutputDirection(original_direction)
    resampler.SetOutputOrigin(original_origin)
    resampler.SetTransform(sitk.Transform())
    resampler.SetDefaultPixelValue(0)

    if is_label:
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
        # resampler.SetInterpolator(sitk.sitkLabelGaussian)
        # resampler.SetSigma(3.0)   # 控制边界模糊的程度，数值越大越模糊（默认1.0）
    else:
        resampler.SetInterpolator(sitk.sitkBSpline)

    return resampler.Execute(itk_image)

def Resampling_back(output_dict, original_spacing, original_origin, original_direction, padding_info):

    output_dict = output_dict.squeeze(0)  # Remove batch dimension if present
    
    original_origin = original_origin[0]  # Assuming original_origin is a list of lists
    original_direction = original_direction[0]  # Assuming original_direction is a list of lists
    original_spacing = original_spacing[0]  # Assuming original_spacing is a list of lists

    # Remove padding
    print(padding_info)
    [z0, z1]= padding_info[0][0]
    [y0, y1] = padding_info[0][1]
    [x0, x1] = padding_info[0][2]
    if len(output_dict.shape) == 4:
        output_dict = output_dict[:, z0:len(output_dict) - z1, y0:len(output_dict[0]) - y1, x0:len(output_dict[0][0]) - x1]
    else:
        output_dict = output_dict[z0:len(output_dict) - z1, y0:len(output_dict[0]) - y1, x0:len(output_dict[0][0]) - x1]

    # print(f"Output shape after removing padding: {output_dict.shape}")
    original_origin = list(map(float, original_origin))  # Convert to float
    original_direction = list(map(float, original_direction))  # Convert to float
    original_spacing = list(map(float, original_spacing))  # Convert to float
    # Prediction to nii
    pred_nii = Prediction_to_nii(output_dict, (1.0, 1.3, 1.3), original_origin, original_direction)

    resampled_pred = resample_image(pred_nii, original_spacing, is_label=True)  # Resample to target spacing
    resampled_pred = sitk.GetArrayFromImage(resampled_pred)  # Convert back to numpy array
    # Convert to One-Hot
    mask_tensor = torch.from_numpy(resampled_pred).long()  # Convert to tensor
    one_hot_mask = torch.nn.functional.one_hot(mask_tensor, num_classes=2).permute(3, 0, 1, 2)  # Convert to one-hot encoding and permute to [C, D, H, W]
    
    return one_hot_mask  # Return the one-hot encoded mask tensor

In [9]:
# from utils import Resampling_back
import json
import os
import numpy as np
from torch import tensor

Test_Path = "/data/hdc/jincan/long_seg/MICCAI25/Resampling/val"
dir_list = os.listdir(Test_Path)
for idx in range(len(dir_list)):
    image_path = os.path.join(Test_Path, dir_list[idx], "GED4.npy")
    image_np = np.load(image_path)
    mask_path = os.path.join(Test_Path, dir_list[idx], "mask_GED4.npy")
    mask_np = np.load(mask_path)
    print(f"The original shape is {image_np.shape}, {mask_np.shape}")
    metadata_path = os.path.join(Test_Path, dir_list[idx], "metadata.json")
    metadata = json.load(open(metadata_path, "r"))
    image_np = np.expand_dims(image_np, axis=0)  # Add batch dimension
    original_spacing = [metadata["original_spacing"]]
    # print(f"The original spacing is {original_spacing}")
    original_origin = [metadata["original_origin"]]
    original_direction = [metadata["original_direction"]]
    padding_info = [metadata["padding_info"]]

    # onehot = np.eye(2)[image_np]       # shape: [D, H, W, C]
    # onehot = onehot.transpose(3, 0, 1, 2)
    image_np = tensor(image_np)  # Convert to tensor if using PyTorch, otherwise keep as numpy array
    image_np = Resampling_back(image_np, original_spacing, original_origin, original_direction, padding_info)
    print(f"The resampled shape is {image_np.shape}")

The original shape is (180, 240, 380), (64, 520, 640)
[[[4, 4], [1, 1], [0, 0]]]
The resampled shape is torch.Size([2, 64, 521, 640])
The original shape is (180, 240, 380), (72, 260, 320)
[[[7, 7], [1, 1], [0, 0]]]
The resampled shape is torch.Size([2, 72, 261, 320])
The original shape is (160, 240, 380), (64, 260, 320)
[[[6, 6], [1, 1], [0, 0]]]
The resampled shape is torch.Size([2, 64, 261, 320])
The original shape is (200, 240, 380), (80, 260, 320)
[[[7, 8], [1, 1], [0, 0]]]
The resampled shape is torch.Size([2, 80, 261, 320])
The original shape is (180, 240, 380), (72, 260, 320)
[[[7, 7], [1, 1], [0, 0]]]
The resampled shape is torch.Size([2, 72, 261, 320])
The original shape is (140, 160, 300), (59, 169, 250)
[[[2, 2], [3, 3], [1, 2]]]
The resampled shape is torch.Size([2, 59, 169, 250])
The original shape is (180, 240, 380), (64, 520, 640)
[[[4, 4], [1, 1], [0, 0]]]
The resampled shape is torch.Size([2, 64, 521, 640])
The original shape is (160, 300, 380), (38, 320, 320)
[[[7, 7]